# Fetch the URL's from the links:
After inspecting the code I have concluded that the code has links with class name refernce internal, so I am fetching out all the links from the page. So we have recieved a total of 68 links.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

# Function to get page source after JavaScript execution
def get_page_source(driver, url):
    driver.get(url)
    time.sleep(3)  # Wait for the page to load completely
    return driver.page_source

# Function to extract internal links with specific class 'reference internal'
def extract_links(soup, base_url):
    links = set()
    base_domain = urlparse(base_url).netloc
    for a_tag in soup.find_all('a', class_='reference internal', href=True):
        href = a_tag['href']
        full_url = urljoin(base_url, href)
        if urlparse(full_url).netloc == base_domain:
            links.add(full_url)
    return links

def main():
    url = "https://docs.nvidia.com/cuda/"

    # Set up Selenium WebDriver with headless option
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)  # Ensure ChromeDriver is installed and in your PATH

    print(f"Extracting links from: {url}")
    page_source = get_page_source(driver, url)
    soup = BeautifulSoup(page_source, "html.parser")

    # Extract links with the specified class
    links = extract_links(soup, url)
    print(f"Found {len(links)} links with class 'reference internal'")

    # Convert the set to a list
    links_list = list(links)
    links_list[0]='https://docs.nvidia.com/cuda'
    # Print the list of links
    for link in links_list:
        print(link)

    # Close the browser
    driver.quit()
    links_list=list(set(links_list))

    return links_list

if __name__ == "__main__":
    links = main()
    # Now `links` contains all the URLs with class 'reference internal'


Extracting links from: https://docs.nvidia.com/cuda/
Found 68 links with class 'reference internal'
https://docs.nvidia.com/cuda
https://docs.nvidia.com/cuda/ampere-compatibility-guide/index.html
https://docs.nvidia.com/cuda/ampere-tuning-guide/index.html
https://docs.nvidia.com/deploy/cuda-compatibility/index.html
https://docs.nvidia.com/nsight-systems/index.html
https://docs.nvidia.com/cuda/cuda-for-tegra-appnote/index.html
https://docs.nvidia.com/cuda/wsl-user-guide/index.html
https://docs.nvidia.com/cuda/debugger-api/index.html
https://docs.nvidia.com/cuda/hopper-tuning-guide/index.html
https://docs.nvidia.com/cuda/nvblas/index.html
https://docs.nvidia.com/cuda/profiler-users-guide/index.html
https://docs.nvidia.com/cuda/incomplete-lu-cholesky/index.html
https://docs.nvidia.com/cuda/cuda-features-archive/index.html
https://docs.nvidia.com/cuda/turing-tuning-guide/index.html
https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html
https://docs.nvidia.com/cuda/cuda-math-api/ind

# Fetch Data from web page:
To get the data from 68 links and store it in a txt file i.e scraped_data.txt file. For that we have initialized 'a' mode in write functions.

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

# Function to get page source after JavaScript execution
def get_page_source(driver, url):
    driver.get(url)
    time.sleep(3)  # Wait for the page to load completely
    return driver.page_source

# Function to scrape data from a single URL
def scrape_data(driver, url):
    print(f"Scraping data from: {url}")
    page_source = get_page_source(driver, url)
    soup = BeautifulSoup(page_source, "html.parser")
    text_content = soup.get_text(separator="\n").strip()
    # Remove excessive newlines
    text_content = re.sub(r'\n+', '\n', text_content)
    return text_content

def main():

    # Set up Selenium WebDriver with headless option
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)  # Ensure ChromeDriver is installed and in your PATH

    # Initialize a list to store all the text content
    all_text_content = []

    # Iterate through each URL and scrape the data
    for url in links:
        text_content = scrape_data(driver, url)
        all_text_content.append(f"URL: {url}\n{text_content}\n\n\n")

    # Write the combined content to a single text file
    with open("scraped_data.txt", 'a', encoding="utf-8") as file:
        for content in all_text_content:
            file.write(content)

    # Close the browser
    driver.quit()

if __name__ == "__main__":
    main()


Scraping data from: https://docs.nvidia.com/cuda/cuda-runtime-api/index.html
Scraping data from: https://docs.nvidia.com/cuda/ampere-compatibility-guide/index.html
Scraping data from: https://docs.nvidia.com/cuda/ampere-tuning-guide/index.html
Scraping data from: https://docs.nvidia.com/deploy/cuda-compatibility/index.html
Scraping data from: https://docs.nvidia.com/nsight-systems/index.html
Scraping data from: https://docs.nvidia.com/cuda/wsl-user-guide/index.html
Scraping data from: https://docs.nvidia.com/cuda/debugger-api/index.html
Scraping data from: https://docs.nvidia.com/cuda/hopper-tuning-guide/index.html
Scraping data from: https://docs.nvidia.com/cuda/nvblas/index.html
Scraping data from: https://docs.nvidia.com/cuda/profiler-users-guide/index.html
Scraping data from: https://docs.nvidia.com/cuda/incomplete-lu-cholesky/index.html
Scraping data from: https://docs.nvidia.com/cuda/cuda-math-api/index.html
Scraping data from: https://docs.nvidia.com/cuda/cuda-quick-start-guide/

# Loading of Text file.
We need to load the txt file using Text loader and initialize Recursive text splitter to create tokens and chunck_overlap

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("scraped_data.txt",encoding='UTF-8') # Replace with the path of the document you want to query from

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap  = 512,
    length_function = len,
)
docs = text_splitter.split_documents(documents)

In [6]:
# Initialize OpenAI key
import os

os.environ["OPENAI_API_KEY"] = 'OpenAi key'

In [7]:
# Initialize Embedding models as we don't have required tokens we have used sentence TransformerEmbeddings and initialized open-source model
from langchain_community.embeddings import SentenceTransformerEmbeddings
from tqdm.autonotebook import tqdm, trange
model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Set-Up milvus Vector data base:
Using langchain we have initialized milvus vector data base and gave the port and host for storing the embeddings, we have also initialized our embedding models here for embedding our data and store it in database

In [8]:
from langchain.vectorstores import Milvus

vector_db = Milvus.from_documents(
    docs,
    model,
    connection_args={"host": "127.0.0.1", "port": "19530"},
)

In [9]:
query = "What is c++"
docs = vector_db.similarity_search(query, k=10)
print(docs[0].page_content) 

The 
Tesla
 and 
Fermi
 architectures are no longer supported starting with CUDA 7.0 and CUDA 9.0, respectively.
3. 
Programming Interface

CUDA C++ provides a simple path for users familiar with the C++ programming language to easily write programs for execution by the device.
It consists of a minimal set of extensions to the C++ language and a runtime library.
The core language extensions have been introduced in 
Programming Model
. They allow programmers to define a kernel as a C++ function and use some new syntax to specify the grid and block dimension each time the function is called. A complete description of all extensions can be found in 
C++ Language Extensions
. Any source file that contains some of these extensions must be compiled with 
nvcc
 as outlined in 
Compilation with NVCC
.
The runtime is introduced in 
CUDA Runtime


In [10]:
# Initialize memorry buffer to use them for follow-up questions
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [11]:
# Initialize model using OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [ ]:
# Initialize a chain for Question and answering, we had a module Conversational Retrieval chain which we can use for Question and answering where we use our vector db as retriver chain and use th
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever(),
    memory=memory)

In [ ]:
query = "Summerize the data"
result = chain({"question": query})
result["answer"]